# Table of Content
1. Importing the library
2. Loading the Dataset
3. Null/NaN values Treatment
4. Exploratory Data Analysis.
    - Best category by total numebr of Orders
    - Best Category by completion of order
    - Worst category by not completed order
    - Compare the completed and not completed orders by category
    - Categories by number of order Per Year
    - Number of orders by category by month    

# Interesting Facts *(outcome of EDA)*:

* Men Fashion has more number of orders than women Fashion :-)
* Spike in orders in November for each category, resaon might be Black Friday and 11.11 Sales.
* Spike in May for each category is not I understand. In May we donot have any sales or promotions.
* 2017 is busiest year for most of categories sales.

# Exploratory Data Analysis

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/pakistans-largest-ecommerce-dataset/Pakistan Largest Ecommerce Dataset.csv');
data

# Subset of data needed for Best selling category analysis

In [ ]:
sub_columns = ['status', 'created_at', 'price', 'qty_ordered', 'category_name_1', 'grand_total', 'discount_amount', 'Working Date', 'BI Status', 'Year', 'Month']
data_sub = data[sub_columns]
data_sub

# Treatment of Null or NaN

* **Treament of rows having all columns as NaN**

In [ ]:
# there are some rows in which all columns are null need to drop all those rows
data_sub.dropna(how='all', inplace=True)

#null_status_rows = pd.isnull(data_sub.status)
data_sub.sort_index(ascending=False)

* **Status null treatment**
> After removing all rows having nan in all column, only 15 are rows in which status is nullwe use mode to all 15 nulls. Mode value is completed.

In [ ]:
data_sub.status.fillna(data_sub.status.mode()[0], inplace=True)

* **Category_name_1 null treatment**

In [ ]:
data_sub.isnull().sum()

* **Category name 1 has 164 null, for time begin, we fill all NaN with mode value**
> Mode value is 'Mobiles & Tablets'

In [ ]:
data_sub.category_name_1.fillna(data_sub.category_name_1.mode()[0], inplace=True)

In [ ]:
data_sub

**Get all unique categories**

In [ ]:
list_of_categories = data_sub['category_name_1'].unique()
total_categories = len(list_of_categories)
print(f'Total categories {total_categories}')

# Best category by total numebr of Orders

In [ ]:
categories_by_order_count = data_sub.groupby('category_name_1').size().reset_index(name='Number of Orders').sort_values('Number of Orders',ascending=False)

bar = px.bar(categories_by_order_count, y='Number of Orders', x='category_name_1',
             title='Number of orders by Category',
             hover_data=['category_name_1'], labels={'category_name_1':'Category name'})
bar.show()

# Best Category by completion of order

In [ ]:
data_sub['status'].unique()

* **Considering order statuses 'complete', 'paid', 'received' as completed**

In [ ]:
completed_orders = data_sub[(data_sub['status'] == 'complete') | (data_sub['status'] == 'paid') | (data_sub['status'] == 'received')]
completed_orders_by_category_count = completed_orders.groupby('category_name_1').size().reset_index(name='Number of Orders').sort_values('Number of Orders', ascending=False)

pie = px.bar(completed_orders_by_category_count, y='Number of Orders', x='category_name_1',
             title='Number of completed orders by Category',
             hover_data=['category_name_1'], labels={'category_name_1':'Category name'})
pie.show()

# Worst category by not completed order

In [ ]:
not_completed_orders = data_sub[(data_sub['status'] != 'complete') & (data_sub['status'] != 'paid') & (data_sub['status'] != 'received')]
not_completed_orders_by_category_count = not_completed_orders.groupby('category_name_1').size().reset_index(name='Number of Orders').sort_values('Number of Orders', ascending=False)

bar = px.bar(not_completed_orders_by_category_count, y='Number of Orders', x='category_name_1',
             title='Number of completed orders by Category',
             hover_data=['category_name_1'], labels={'category_name_1':'Category name'})
bar.update_traces(marker_color='red')
bar.show()

# Compare the completed and not completed orders by category

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Completed', x=completed_orders_by_category_count['category_name_1'], y=completed_orders_by_category_count['Number of Orders']),
    go.Bar(name='Not Completed', x=not_completed_orders_by_category_count['category_name_1'], y=not_completed_orders_by_category_count['Number of Orders'])
])
fig.update_layout(barmode='group')
fig.show()

# Categories by number of completed order Per Year

In [ ]:
orders_by_category_year = completed_orders.groupby(['category_name_1', 'Year']).size().reset_index(name='Number of Orders').sort_values(['Number of Orders', 'Year'], ascending=False)
bar = px.bar(orders_by_category_year, y='Number of Orders', x='category_name_1',
             title='Number of orders by Category Per Year', color='Year',
             hover_data=['category_name_1'], labels={'category_name_1':'Category name'})
bar.show()

# Number of orders by category by month

In [ ]:
data_sub.category_name_1.unique()

In [ ]:
orders_by_category_month = completed_orders.groupby(['category_name_1', 'Month']).size().reset_index(name='Number of Orders').sort_values('Month')

mobile_tablets = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Mobiles & Tablets']

men_fashion = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Men\'s Fashion']

others = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Others']

women_fashion = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Women\'s Fashion']

appliances = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Appliances']

superstore = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Superstore']

entertainment = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Entertainment']

beauty_grooming = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Beauty & Grooming']

home_living = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Home & Living']

computing = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Computing']

soghaat = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Soghaat']

kids_baby = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Kids & Baby']

n = orders_by_category_month[orders_by_category_month['category_name_1'] == '\\N']

health_sports = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Health & Sports']

school_education = orders_by_category_month[orders_by_category_month['category_name_1'] == 'School & Education']

books = orders_by_category_month[orders_by_category_month['category_name_1'] == 'Books']

fig = go.Figure()
fig.add_trace(go.Scatter(x=mobile_tablets['Month'], y=mobile_tablets['Number of Orders'],
                    mode='lines+markers',
                    name='Mobiles & Tablets'))

fig.add_trace(go.Scatter(x=men_fashion['Month'], y=men_fashion['Number of Orders'],
                    mode='lines+markers',
                    name='Men\'s Fashion'))

fig.add_trace(go.Scatter(x=others['Month'], y=others['Number of Orders'],
                    mode='lines+markers',
                    name='Others'))

fig.add_trace(go.Scatter(x=women_fashion['Month'], y=women_fashion['Number of Orders'],
                    mode='lines+markers',
                    name='Women Fashion'))

fig.add_trace(go.Scatter(x=appliances['Month'], y=appliances['Number of Orders'],
                    mode='lines+markers',
                    name='Appliances'))

fig.add_trace(go.Scatter(x=superstore['Month'], y=superstore['Number of Orders'],
                    mode='lines+markers',
                    name='Superstore'))

fig.add_trace(go.Scatter(x=entertainment['Month'], y=entertainment['Number of Orders'],
                    mode='lines+markers',
                    name='Entertainment'))

fig.add_trace(go.Scatter(x=beauty_grooming['Month'], y=beauty_grooming['Number of Orders'],
                    mode='lines+markers',
                    name='Beauty grooming'))

fig.add_trace(go.Scatter(x=kids_baby['Month'], y=kids_baby['Number of Orders'],
                    mode='lines+markers',
                    name='Kids and Baby'))

fig.add_trace(go.Scatter(x=home_living['Month'], y=home_living['Number of Orders'],
                    mode='lines+markers',
                    name='Home Living'))

fig.add_trace(go.Scatter(x=soghaat['Month'], y=soghaat['Number of Orders'],
                    mode='lines+markers',
                    name='Soghaat'))

fig.add_trace(go.Scatter(x=n['Month'], y=n['Number of Orders'],
                    mode='lines+markers',
                    name='N'))

fig.add_trace(go.Scatter(x=health_sports['Month'], y=health_sports['Number of Orders'],
                    mode='lines+markers',
                    name='Health Sports'))

fig.add_trace(go.Scatter(x=school_education['Month'], y=school_education['Number of Orders'],
                    mode='lines+markers',
                    name='School Education'))

fig.add_trace(go.Scatter(x=books['Month'], y=books['Number of Orders'],
                    mode='lines+markers',
                    name='Books'))

fig.show()

# Please Upvote if you find the notebook interesting